In [37]:
%autoreload 2

sys.path.append('../data')
from data_loading import *

from Model import *

fname = "../data/up_down/100GEV-downquark-seed1.txt"

jets = jets_from_pythia_txt(fname)
print("Number of jets in file: ", len(jets))

Number of jets in file:  10000


In [41]:
from JetImages import sort_particles_into_bins

eta_bins, phi_bins, particle_idxx = sort_particles_into_bins(jets[0])

In [46]:
phi_bins[1] - phi_bins[0], eta_bins[1] - eta_bins[0]

(0.02400000000000002, 0.02400000000000002)